In [16]:
from scipy.spatial import Voronoi, voronoi_plot_2d, Delaunay, cKDTree
import pypcd.pypcd as pypcd
import numpy as np
import open3d as o3d
import time

In [7]:
stadler_blue =(30, 90, 154)
stadler_yellow = (255, 189, 0)
stadler_grey = (192,192,192)

In [21]:
# def upsample(pc, n):
    
#     start = time.time()
    
#     vor = Voronoi(pc[:, :3])
#     tree = cKDTree(pc[:, :3])
#     nearest_points = tree.query(vor.vertices)
#     distances = nearest_points[0]
#     indices = nearest_points[1]
#     sorted_indices = np.argsort(distances)
#     sorted_distances = distances[sorted_indices]
#     clostest_indices = sorted_indices[:n]
#     clostest_points = vor.vertices[clostest_indices]
#     new_points = np.concatenate([pc[:, :3], clostest_points], axis=0)
    
#     end = time.time()
    
    
#     print("Time: ", end - start)
        
#     return new_points

In [22]:
# def upsample_open3d(pc, max_points):
    
#     start = time.time()
    
#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(pc[:, :3])
#     alpha = 0.2
#     mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
#     # mesh.compute_vertex_normals()
#     pcd = mesh.sample_points_poisson_disk(max_points)
    
#     print("Time taken: ", time.time() - start)
#     return np.asarray(pcd.points)

In [2]:
def upsample_ball_pivoting(pc, max_points):
    
    start = time.time()
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pc[:, :3])
    pcd.estimate_normals()
    distances = pcd.compute_nearest_neighbor_distance()
    avg_dist = np.mean(distances)
    radius = avg_dist*3
    # print("Radius: ", radius)
    # pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=30))
    bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector([radius, radius * 2, radius * 4]))
    pcd = bpa_mesh.sample_points_poisson_disk(max_points)
    
    # print("Time taken: ", time.time() - start)
    return np.asarray(pcd.points)

In [24]:
# def upsample_open3d(pc, max_points):
#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(pc[:, :3])
#     pcd.normals = o3d.utility.Vector3dVector(np.zeros_like(pc[:, :3]))
#     pcd.estimate_normals()
#     mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9, width=0, scale=1.1, linear_fit=False)[0]
#     mesh.compute_vertex_normals()
#     pcd = mesh.sample_points_poisson_disk(max_points)
#     return np.asarray(pcd.points)

In [3]:
# folder = '/home/cws-ml-lab/mmdetection3d_for_rail/data/robosense_m1_plus_sequences/kitti_gt_database'
folder = '/home/cws-ml-lab/mmdetection3d_for_rail/data/robosense_m1_plus_sequences/kitti_gt_database'

In [4]:
lower_limit = 50
upper_limit = 256

In [ ]:
# iterate through all files in the folder
import os

all_files = os.listdir(folder)
# shuffle the files

all_files = ['/home/cws-ml-lab/mmdetection3d_for_rail/data/robosense_m1_plus_sequences/kitti_gt_database/781_Cyclist_0.bin']

np.random.shuffle(all_files)


for filename in all_files:
    
    print("Processing file: ", filename)
    
    # read the point cloud
    pc = np.fromfile(os.path.join(folder, filename), dtype=np.float32).reshape(-1, 4)
    
    if pc.shape[0] < lower_limit:
        continue
    
    if pc.shape[0] > upper_limit:
        continue
    
    try:

        num_points_for_interp = 256 - pc.shape[0]
        new_pc = upsample_ball_pivoting(pc, num_points_for_interp)
        
        # make all points black
        
        print("Original point cloud")
        print("Nump points: ", pc.shape[0])
        
        # plot once before upsampling
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(pc[:, :3])
        pcd.colors = o3d.utility.Vector3dVector(np.zeros((pc.shape[0], 3)))
        o3d.visualization.draw_geometries([pcd])
        
        # pc and new_pc are of size (N, 3) and (M, 3) respectively
        
        new_pc = np.concatenate([pc[:, :3], new_pc], axis=0)
        
        color = np.zeros((new_pc.shape[0], 3))
        color[pc.shape[0]:, :] = [30, 90, 154]
        color[:pc.shape[0], :] = [0, 0, 0]
        
        # increase point size
        
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(new_pc)
        pcd.colors = o3d.utility.Vector3dVector(color.astype(np.float64)/255.0)
        o3d.visualization.draw_geometries([pcd])
        
    except Exception as e:
        print("Error: ", e)
        continue

In [30]:
# point_cloud_path =  '/home/cws-ml-lab/mmdetection3d_for_rail/data/kitti_cls/kitti_gt_database/781_Cyclist_0.bin'
point_cloud_path = '/home/cws-ml-lab/mmdetection3d_for_rail/data/robosense_m1_plus_sequences/kitti_gt_database/781_Cyclist_0.bin'

In [38]:
# load the point cloud
pc = np.fromfile(point_cloud_path, dtype=np.float32).reshape(-1, 4)

color = np.zeros((pc.shape[0], 3))

# visualize the point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc[:, :3])
o3d.visualization.draw_geometries([pcd])


In [17]:
# upsample

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc[:, :3])
pcd.estimate_normals()
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = avg_dist*3
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector([radius, radius * 2, radius * 4]))
# set color of the mesh
bpa_mesh.paint_uniform_color((0.117, 0.352, 0.603))

# viualize the mesh
o3d.visualization.draw_geometries([bpa_mesh])